# Monte Carlo

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
import numpy as np
from scipy import integrate

In [ ]:
n = 10 ** 4

## Rejection Sampling

Sampling from the uniform distribution $X\sim U(a, b)$.

In [ ]:
np.random.uniform(0, 1)

Sampling from an arbitrary function $X\sim\rho$, where $\rho$ is a density defined on the finite, connected domain $(a, b)$. This uses principles of *rejection sampling*.

In [ ]:
def sample_arbitrary_function(rho, a, b, 
                              n = 1, 
                              phi = lambda x:x,
                              sampling = np.random.uniform):
    # phi is some operation.
    
    if n == 1:
        rejected = True
        while rejected:
            
            # different sampling methods can be chosen here.
            x = sampling(a, b)
            
            # case of acceptance.
            if rho(x) >= np.random.uniform(0, 1):
                rejected = False
                return phi(x)
            
            pass
        pass
    else:
        return np.array([sample_arbitrary_function(rho, a, b, n = 1, phi = phi) for _ in range(n)])
    pass

Demonstration with the sine function, defined on $(0, \pi)$.

In [ ]:
p = lambda x:1/2*np.sin(x)

In [ ]:
sample_arbitrary_function(p, 0, np.pi)

In [ ]:
a, b = 0, np.pi

solution = sample_arbitrary_function(p, a, b, n = n)

In [ ]:
plt.hist(solution, 50, density = True)

xx = np.linspace(0, np.pi, 100)
plt.plot(xx, p(xx))

plt.title('Histogram')

plt.show()

## Monte Carlo Approximation

Let $x_1, \cdots, x_n\sim p$, then the sample mean $\hat{\mu}_n = \dfrac{1}{n}\sum_{i = 1}^n\phi(x_i)$ is a basic Monte Carlo estimator of $\mathbb{E}\phi(x)$.

## Importance sampling

Let $X\sim p$. Then the expectation $\mathbb{E}\phi(x) = \int_\Omega\phi(x)p(x)dx\approx\dfrac{1}{n}\sum_{i = 1}^n\phi(x_i)$. Let $q(x)$ be a proposal density such that $q(x) = 0$ if and only if $p(x) = 0$ (absolute continuity). Then $\mathbb{E}\phi(x) \approx\dfrac{1}{n}\sum_{i = 1}^n\phi(y_i)w(y_i)$, where $w(y) = \dfrac{p(y)}{q(y)}$ and $Y\sim q$.

**Principle**: choose $q(x)$ such that $q(x)\propto\vert\phi(x)\vert p(x)$, i.e. $q$ places more weight on regions where $\vert\phi(x)\vert p(x)$ is large.

However, usually we can't find such an exact $q$. Because if we did, we would have also found the partition function of $\phi(x)p(x)$, and that would be the desired expectation.

### Demonstration

From the density $p(x) = \dfrac{1}{2}\sin(x)$ we sample $X$. Let $\phi(x) = x^2$. We first find the expectation $\mathbb{E}\phi(x)$ analytically, and yield $\dfrac{1}{2}(\pi^2-4)$. Next, we sample the approximation directly.

In [ ]:
theoretical_expectation = 1/2*(np.pi**2 - 4)
print(theoretical_expectation)

In [ ]:
phi = lambda x:np.power(x, 2)

direct_sample = sample_arbitrary_function(p, a, b, n = n, phi = phi)
plt.hist(direct_sample, 50, density = True)
plt.show()

direct_sample_mean = np.average(direct_sample)

print('Sample from direct average: {}'.format(direct_sample_mean))

Let $q(x) \propto \vert\phi(x)\vert p(x) = \dfrac{x^2}{2}\sin(x)$ be a density function, then $q =\dfrac{\sin(x)}{2Z}$, where the partition function $Z = \dfrac{\pi^2 - 4}{2}$, is a legal density function. We can varify absolute continuity.

In [ ]:
q = lambda x:np.power(x, 2)/(np.power(np.pi, 2) - 4)*np.sin(x)

new_sampling = lambda a, b:sample_arbitrary_function(q, a, b)
sample = sample_arbitrary_function(q, a, b, n = n, sampling = new_sampling)

plt.hist(sample, 50, density = True)

xx = np.linspace(0, np.pi)
plt.plot(xx, q(xx))

plt.title('Map of proposal density function')
plt.show()

Recall that we defined $q(x) = Z^{-1}\phi(x)p(x)$. Therefore, $\mathbb{E}\phi(x) = \dfrac{1}{n}\sum_{i = 1}^n\phi(x_i)w(x_i) = \dfrac{\pi^2 - 4}{2}$ is trivially obtained. In this case, our sampling does not influence the outcome of the result, since $\phi w$ is a constant function.

## Markov Chain Monte Carlo

A Markov chain which is irreducible, has a stationary distribution $\pi$, and is aperiodic, is an *ergodic* Markov chain.

I think [Wikipedia](https://en.wikipedia.org/wiki/Markov_chain) explains aperiodicity best:

> A state $i$ has period $k$, if any return to state $i$ must occur in multiples of $k$ time steps. [...] If $k = 1$, then the state is said to be *aperiodic*.

Also refer to this [post](https://math.stackexchange.com/questions/1227869/period-of-a-markov-chain-why-is-this-one-aperiodic) on my shared misconception with the asker.

### Markov chain model algorithm

We design an algorithm modeling the movement of a particle in a Markov chain with an $n\times n$ transition matrix $M$.

Suppose we have a Markov matrix $M = \begin{bmatrix}.25 & .75\\.4 & .6\end{bmatrix}$, then a cumulative probability matrix is $C = \begin{bmatrix}.25 & 1\\.4 & 1\end{bmatrix}$

In [ ]:
def markov_chain_cumulative_matrix(transition_matrix):
    n = len(transition_matrix)
    solution = [[None for _ in range(n)] for _ in range(n)]
    
    for i in range(n):
        for j in range(n):
            if j == 0:
                solution[i][0] = transition_matrix[i][0]
                pass
            else:
                solution[i][j] = solution[i][j-1] + transition_matrix[i][j]
                pass
            pass
        pass
    return np.array(solution)

A demonstration of the cumulative matrix.

In [ ]:
markov_chain_cumulative_matrix(np.array([
    [.25, .75],
    [.4, .6]
]))

The reflective random walk matrix.

In [ ]:
reflective_random_walk = np.array([
    [.0, 1., 0., 0., .0, .0],
    [.5, .0, .5, .0, .0, .0],
    [.0, .5, .0, .5, .0, .0],
    [.0, .0, .5, .0, .5, .0],
    [.0, .0, .0, .5, .0, .5],
    [.0, .0, .0, .0, 1., .0]
])

markov_chain_cumulative_matrix(reflective_random_walk)

In [ ]:
def markov_chain_sample(cumulative_matrix, current_value, dim):
    # extract relevant probabilities.
    
    cumulative_probabilities = cumulative_matrix[current_value]
    
    # sample u.
    u = np.random.uniform(0, 1)
    
    # find greatest upper bound.
    for j in range(dim):
        if u <= cumulative_probabilities[j]:
            return j
        pass
    pass

Sampling the next value in a Markov chain with current value $x$.

In [ ]:
markov_chain_sample(markov_chain_cumulative_matrix(reflective_random_walk), 2, 6)

In [ ]:
def markov_chain(transition_matrix, initial_state, n):
    cumulative_matrix = markov_chain_cumulative_matrix(transition_matrix)
    dim = len(transition_matrix)
    
    # standardize arguments.
    transition_matrix = np.array(transition_matrix)
    
    # construct solution space.
    solution = [None for _ in range(n)]
    
    # sample from the Markov chain.
    for i in range(n):
        # initial condition.
        if i == 0:
            solution[0] = initial_state
            pass
        
        else:
            current_value = solution[i-1]
            solution[i] = markov_chain_sample(cumulative_matrix, current_value, dim)
        pass
    
    return np.array(solution)

In [ ]:
transition_matrix = np.array([
    [.25, .75],
    [.4, .6]
])
initial_state = 0
solution = markov_chain(transition_matrix, initial_state, n)

In [ ]:
left_of_first_bin = -0.5
right_of_last_bin = 2.5

plt.hist(solution, np.arange(left_of_first_bin, right_of_last_bin, 1), density = True)
plt.title('Markov chain state histogram')
plt.show()

### Reflective random walks

A reflective random walk is a Markov process. Particles in the random walk are bounded on both sides, and move randomly in between.

In [ ]:
initial_state = 0
solution = markov_chain(reflective_random_walk, initial_state, n)

In [ ]:
left_of_first_bin = -.5
right_of_last_bin = 6.5

plt.hist(solution, np.arange(left_of_first_bin, right_of_last_bin, 1), density = True)
plt.title('Reflective random walk state histogram')
plt.show()

### Detailed balance

A probability mass function $\pi$ on $\mathcal{X}$ satisfies *detailed balance* with respect to a transition matrix $T = (T_{ab})$, if $\pi_aT_{ab} = \pi_bT_{ba}$ for all $a, b\in\mathcal{X}$. Furthermore, detailed balance implies that $\pi$ is a stationary distribution, i.e. $\pi T = \pi$. Detailed balance is also known as reversibility.

## Metropolis Algorithm

Consider a pmf $\pi$ on countable $\mathcal{X}$ and an observable $\phi:\mathcal{X}\to\mathbb{R}$. Our goal is to approximate $\mathbb{E}\phi(x)$ where $x\sim\pi$, and where directly sampling is prohibitively expensive.

Let $Q$ be a symmetric, stochastic matrix, termed the *proposal matrix*, such that $Q = (Q_{ab}: a, b\in\mathcal{X})$. Let $\tilde{\pi} = Z\pi$. Computing $\tilde{\pi}(x)$ without having to compute $Z$ is sufficient in implementing the Metropolis algorithm.

The algorithm is described as follows:

1. Choose a proposal matrix $Q$,
2. Set initial value $x_0\sim\mathcal{X}$,
3. For $i = 0, \cdots, n - 1$, sample $x\sim Q_{x_ix}$ and $u\sim U(0, 1)$. If $u < \dfrac{\tilde{\pi}(x)}{\tilde{\pi}(x_i)}$, then $x_{i + 1} = x$. Otherwise, $x_{i + 1} = x_i$.

**Principle**: it is found that the identity matrix $I$ is not a suitable proposal matrix.

### Modified reflective walk algorithm

Let a Markov chain be described by the transition matrix, $T = \begin{bmatrix}.5 & .5 & 0\\.25 & .5 & .25\\0 & .5 & .5\end{bmatrix}$. The matrix is irreducible. Next, there exists a stationary pmf $\pi = \begin{bmatrix}.25 & .5 & .25\end{bmatrix}$. Furthermore, the period for any state is $1$. Therefore, this matrix is ergodic.

In [ ]:
reflective_walk_modified = np.array([
    [.5, .5, .0],
    [.25, .5, .25],
    [.0, .5, .5]
])
pi = np.array([[.25, .5, .25]])

# we can confirm that pi is indeed stationary with respect to T.
np.dot(pi, reflective_walk_modified)

In [ ]:
# initial state.
initial_state = 0
solution = markov_chain(reflective_walk_modified, initial_state, n)

In [ ]:
left_of_first_bin = -.5
right_of_last_bin = 3.5
plt.hist(solution, np.arange(left_of_first_bin, right_of_last_bin, 1), density = True)
plt.title('Modified walk distribution')
plt.show()

Let our proposal matrix be $\begin{bmatrix}.5 & .25 & .25\\.25 & .5 & .25\\.25 & .25 & .5\end{bmatrix}$. We can let $\tilde{\pi}$ be any pmf proportional to $\pi$, so let's just have $\tilde{\pi} = \begin{bmatrix} 1 & 2 & 1\end{bmatrix}$.

In [ ]:
proposal_matrix = np.array([
    [.5, .25, .25],
    [.25, .5, .25],
    [.25, .25, .5]
])

pi_tilde = np.array([[1, 2, 1]])

In [ ]:
initial_state = 0
cumulative_matrix = markov_chain_cumulative_matrix(proposal_matrix)
dim = len(proposal_matrix)

# solution set.
solution = [None for _ in range(n)]

# compute solution.
for i in range(n):
    if i == 0:
        solution[0] = initial_state
        pass
    else:
        current_value = solution[i-1]
        x = markov_chain_sample(cumulative_matrix, current_value, dim)
        u = np.random.uniform(0, 1)

        if u <= pi_tilde[0][x] / pi_tilde[0][current_value]:
            solution[i] = x
            pass
        else:
            solution[i] = current_value
            pass
        pass
    pass

In [ ]:
left_of_first_bin = -.5
right_of_last_bin = 3.5

plt.hist(solution, np.arange(left_of_first_bin, right_of_last_bin, 1), density = True)
plt.title('Modified walk distribution (Metropolis)')
plt.show()

## Metropolis Hastings Algorithm

Let $f(x)$ be proportional to the target density up to a normalizing constant, and set the current value $x^j$. We sample from the proposal density $x^*\sim q(x\vert x^j)$. The probability that $x^*$ is accepted is $\rho(x^j, x^*) = \min\bigg\{1, \dfrac{f(x^*)}{f(x^j)}\dfrac{q(x^j\vert x^*)}{q(x^*\vert x^j)}\bigg\}$. If $x^*$ is accepted, then $x^{j+1} = x^*$; otherwise $x^{j+1} = x^j$.

**Demonstration**: consider the truncated normal distribution $N(5, 9)I(1\leq x\leq 6)$. The unnormalized target density is $f(x)\propto\exp(-(x-5)^2/18)I(1\leq x\leq 6)$, and we choose a proposal distribution $q(x\vert x^j) = N(x\vert x^j, 1)$ and let $x^0 = 5$. (Video: [Niemi](https://www.youtube.com/watch?v=VGRVRjr0vyw))

In [ ]:
normalization_constant = integrate.quad(lambda x:np.exp(-(x-5)**2/18)*(1<x<6), 1, 6)[0]
def target_dist(x):
    if isinstance(x, np.ndarray):
        return np.array([target_dist(_x) for _x in x])
    return np.exp(-(x-5)**2/18)*(1<x<6)/normalization_constant

proposal_dist = lambda x, mu:np.exp(-(x-mu)**2)
proposal_dist_sample = lambda x:np.random.normal(x, 1)

In [ ]:
def metropolis_hastings_random_walk(m):
    # this is a demonstration function.
    # m is the number of trials.
    x_init = 5
    solution = [None for _ in range(m)]
    for i in range(m):
        if i == 0:
            solution[0] = x_init
            pass
        else:
            current_value = solution[i-1]
            sampled_value = proposal_dist_sample(current_value)
            acceptance_probability = min(1, (target_dist(sampled_value)*proposal_dist(current_value, sampled_value))/(target_dist(current_value)*proposal_dist(sampled_value, current_value)))
            u = np.random.uniform(0, 1)
            # conditions for accept.
            if u <= acceptance_probability:
                solution[i] = sampled_value
                pass
            # conditions for reject.
            else:
                solution[i] = current_value
                pass
            pass
        pass
    return solution

Comparison of the Metropolis-Hastings algorithm at different densities.

In [ ]:
xx = np.linspace(1, 6, 100)

s = 6 # number of densities.
i = 1 # initial subplot index

f, axs = plt.subplots(2,2,figsize=(6.4 * 2, 4.8 * s))

for k in range(1, s + 1):
    solution = metropolis_hastings_random_walk(10 ** k)
    
    plt.subplot(s, 2, i)
    plt.plot(solution)
    plt.title('Evolution ({} iterations)'.format(10 ** k))
    i += 1
    
    plt.subplot(s, 2, i)
    plt.hist(solution, 100, density = True)
    plt.step(xx, target_dist(xx), label = 'target')
    plt.title('Histogram ({} iterations)'.format(10**k))
    plt.legend()
    i += 1
    pass

plt.show()